In [1]:
import math
import pandas as pd
import networkx as nx
from itertools import combinations

edges = pd.read_csv("./駅データ/join20240426.csv")
stations = pd.read_csv("./駅データ/station20240426.csv")
line = pd.read_csv("./駅データ/line20240426.csv", usecols=["line_cd", "line_name"])

In [2]:
stations = stations.merge(line, how='left')

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    # 緯度経度から2点間の距離を求める
    
    # 地球の半径（キロメートル）
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [4]:
def get_station_cd(station_name):
    station_cd = stations.loc[stations["station_name"] == station_name, "station_cd"]
    if not station_cd.empty:
        return station_cd.iloc[0]
    else:
        raise ValueError(f"No station found with name {station_name}")

def simplify_path(path):
    if not path:
        return []
    simplified_path = [path[0]]
    current_line = G.nodes[path[0]]['line_cd']
    for station in path[1:]:
        next_line = G.nodes[station]['line_cd']
        if next_line != current_line:
            simplified_path.append(station)
            current_line = next_line
    if path[-1] != simplified_path[-1]:
        simplified_path.append(path[-1])
    return simplified_path


In [5]:
edges = edges.merge(stations.loc[:,["station_cd", "lon", "lat"]], how='left', left_on='station_cd1', right_on='station_cd', suffixes=('', '_1'))
edges.drop(columns=['station_cd'], inplace=True)
edges.rename(columns={'lon': 'lon1', 'lat': 'lat1'}, inplace=True)

edges = edges.merge(stations.loc[:,["station_cd", "lon", "lat"]], how='left', left_on='station_cd2', right_on='station_cd', suffixes=('', '_2'))
edges.drop(columns=['station_cd'], inplace=True)
edges.rename(columns={'lon': 'lon2', 'lat': 'lat2'}, inplace=True)
edges = edges.dropna()

edges['distance'] = edges.apply(lambda row: haversine(row["lat1"], row["lon1"], row["lat2"], row["lon2"]), axis=1)
# 距離の逆数(0除算対策の0.1)
edges['weight'] = edges['distance'].apply(lambda x: 1 / (x + 0.1))

In [6]:
G = nx.Graph()

for idx, row in stations.iterrows():
    G.add_node(row['station_cd'], name=row['station_name'] + " " + row["line_name"], lat=row['lat'], lon=row['lon'], line_cd=row['line_cd'])

for idx, row in edges.iterrows():
    src_cd = row['station_cd1']
    dst_cd = row['station_cd2']
    weight = row["weight"]
    G.add_edge(src_cd, dst_cd, weight=weight)

grouped_stations = stations.groupby('station_g_cd')

for group_cd, group in grouped_stations:
    station_pairs = combinations(group['station_cd'], 2)
    for src_cd, dst_cd in station_pairs:
        G.add_edge(src_cd, dst_cd, weight=edges['weight'].max() * 2)


In [7]:
def print_shortest_path(source_name, target_name):
    try:
        source = get_station_cd(source_name)
        target = get_station_cd(target_name)
        
        shortest_path_cd = nx.shortest_path(G, source=source, target=target, weight='weight')
        simplified_path_cd = simplify_path(shortest_path_cd)
        
        shortest_path_names = [G.nodes[data]['name'] for data in shortest_path_cd]
        simplified_path_names = [G.nodes[data]['name'] for data in simplified_path_cd]
        
        print("最短経路（駅名）\n", " ->\n".join(simplified_path_names))
    
    except nx.NetworkXNoPath:
        print("経路が見つかりません。")


In [10]:
source_name = "稚内"
target_name = "鹿児島"

print_shortest_path(source_name, target_name)

最短経路（駅名）
 稚内 JR宗谷本線 ->
旭川 JR函館本線(小樽～旭川) ->
札幌 北海道新幹線 ->
新青森 東北新幹線 ->
東京 東海道新幹線 ->
新大阪 山陽新幹線 ->
博多 九州新幹線 ->
鹿児島中央 JR鹿児島本線(川内～鹿児島) ->
鹿児島 JR鹿児島本線(川内～鹿児島)


In [11]:
source_name = "横浜"
target_name = "城崎温泉"

print_shortest_path(source_name, target_name)

最短経路（駅名）
 横浜 JR東海道本線(東京～熱海) ->
品川 東海道新幹線 ->
京都 嵯峨野線 ->
園部 JR山陰本線(園部～豊岡) ->
豊岡 JR山陰本線(豊岡～米子) ->
城崎温泉 JR山陰本線(豊岡～米子)
